In [ ]:
import struct
import numpy as np
import matplotlib.pyplot as plt
import io
from PIL import Image as im
import dct
%matplotlib inline 


file_dir = './lady.dat'
with open(file_dir , 'rb') as f:
    fbuff = io.BufferedReader(f)
    N = len(fbuff.read())

with open(file_dir , 'rb') as f:
    data = struct.unpack('b'* N, f.read(1*N)) 

In [ ]:
# 获得数据
data = np.asarray(data,dtype= np.uint8)
data = data.reshape(256,256)
plt.imshow(data,cmap="gray")  

In [ ]:
rows_blocks = 256//8
cols_blocks = 256//8
blocks_num = rows_blocks * cols_blocks
blocks = np.empty([rows_blocks * cols_blocks, 8 , 8],dtype=np.int32)


for row_blocks in range(rows_blocks):
    for col_blocks in range(cols_blocks):
        blocks[row_blocks * rows_blocks + col_blocks,:,:] = data[8 * row_blocks : 8 * row_blocks + 8, 8 * col_blocks : 8 * col_blocks +8]

plt.imshow(blocks[120],cmap='gray')

In [ ]:
quant_table = np.asarray([[16,11,10,16,24,40,51,61],
                        [12,12,14,19,26,58,60,55],
                        [14,13,16,24,40,57,69,56],
                        [14,17,22,29,51,87,80,62],
                        [18,22,37,56,68,109,103,77],
                        [24,35,55,64,81,104,113,92],
                        [49,64,78,87,103,121,120,101],
                        [72,92,95,98,112,100,103,99]])
ac_encode_table = np.asarray([["1010","00", "01", "100", "1011", "11010", "1111000", "11111000", "1111110110", "1111111110000010", "1111111110000011"],
            ["","1100", "11011", "1111001", "111110110", "11111110110", "1111111110000100", "1111111110000101", "1111111110000110", "1111111110000111", "1111111110001000"],
            ["","11100", "11111001", "1111110111", "111111110100", "1111111110001001", "1111111110001010", "1111111110001011", "1111111110001100", "1111111110001101","1111111110001110"],
            ["","111010", "111110111", "111111110101", "1111111110001111", "1111111110010000", "1111111110010001", "1111111110010010", "1111111110010011", "1111111110010100", "1111111110010101"],
            ["","111011", "1111111000", "1111111110010110", "1111111110010111", "1111111110011000", "1111111110011001", "1111111110011010", "1111111110011011", "1111111110011100", "1111111110011101"],
            ["","1111010", "11111110111", "1111111110011110", "1111111110011111","1111111110100000", "1111111110100001", "1111111110100010", "1111111110100011", "1111111110100100", "1111111110100001"],
            ["","1111011", "111111110110", "1111111110100110", "1111111110100111", "1111111110101000", "1111111110101001", "1111111110101010", "1111111110101011", "1111111110101100", "1111111110101101"],
            ["","11111010", "111111110111", "1111111110101110", "1111111110101111", "1111111110110000", "1111111110110001", "1111111110110010", "1111111110110011", "1111111110110100", "1111111110110101"],
            ["","111111000", "111111111000000", "1111111110110110", "1111111110110111", "1111111110111000", "1111111110111001", "1111111110111010", "1111111110111011", "1111111110111100", "1111111110111101"],
            ["","111111001", "1111111110111110", "1111111110111111", "1111111111000000", "1111111111000001", "1111111111000010", "1111111111000011", "1111111111000100", "1111111111000101", "1111111111000110"],
            ["","111111010", "1111111111000111", "1111111111001000", "1111111111001001", "1111111111001010", "1111111111001011", "1111111111001100", "1111111111001101", "1111111111001110", "1111111111001111"],
            ["","1111111001", "1111111111010000", "1111111111010001", "1111111111010010", "1111111111010011", "1111111111010100", "1111111111010101", "1111111111010110", "1111111111010111", "1111111111011000"],
            ["","1111111010", "1111111111011001", "1111111111011010", "1111111111011011", "1111111111011100", "1111111111011101", "1111111111011110", "1111111111011111", "1111111111100000", "1111111111100001"],
            ["","11111111000", "1111111111100010", "1111111111100011", "1111111111100100", "1111111111100101", "1111111111100110", "1111111111100111", "1111111111101000", "1111111111101001", "1111111111101010"],
            ["","1111111111101011", "1111111111101100", "1111111111101101", "1111111111101110", "1111111111101111", "1111111111110000", "1111111111110001", "1111111111110010", "1111111111110011", "1111111111110100"],
            ["11111111001", "1111111111110101", "1111111111110110", "1111111111110111", "1111111111111000", "1111111111111001", "1111111111111010", "1111111111111011", "1111111111111100", "1111111111111101", "1111111111111110"]],dtype=str)
dc_encode_table = ["00","010", "011","100","101","110","1110","11110","111110","1111110","11111110","111111110"]
zigzag_col = [0,0,1,2,1,0,0,1,2,3,4,3,2,1,0,0,1,2,3,4,5,6,5,4,3,2,1,0,0,1,2,3,4,5,6,7,7,6,5,4,3,2,1,2,3,4,5,6,7,7,6,5,4,3,4,5,6,7,7,6,5,6,7,7]
zigzag_row = [0,1,0,0,1,2,3,2,1,0,0,1,2,3,4,5,4,3,2,1,0,0,1,2,3,4,5,6,7,6,5,4,3,2,1,0,1,2,3,4,5,6,7,7,6,5,4,3,2,3,4,5,6,7,7,6,5,4,5,6,7,7,6,7]

In [ ]:

blocks = np.asarray(blocks - 128.0,dtype=np.float64)
blocks_dct = np.empty([blocks_num,8,8],dtype=np.float64)
for index in range(blocks_num):
    blocks_dct[index,:,:]  = dct.dct8x8(blocks[index]) 
blocks_quant = np.asarray(np.round(blocks_dct / quant_table),dtype=np.int32)
dc_values = blocks_quant[:,0,0].reshape(-1) 
ac_values = blocks_quant


In [ ]:
def dc_encode(value,pre_value):
    global dc_encode_table
    dc_before_code = value - pre_value
    encode_len = 0 if dc_before_code == 0 else int(np.log2(abs(dc_before_code))) + 1 
    encode_preffix = dc_encode_table[encode_len]
    encode = ''
    if dc_before_code > 0:
        encode = encode_preffix + str(bin(dc_before_code))[2:]
    elif dc_before_code < 0:
        original = str(bin(dc_before_code))[3:]
        encode_suffix = ''.join(['0' if x == '1' else '1' for x in original])
        encode = encode_preffix + encode_suffix
    else :
        encode = encode_preffix
    return encode



In [ ]:
def act_ac_encode(zero_num, nozero_value):
        global ac_encode_table
        nozero_value_level = 0 if nozero_value == 0 else int(np.log2(abs(nozero_value))) + 1
        encode = ''
        encode_preffix = ac_encode_table[zero_num,nozero_value_level]
        if nozero_value > 0:
                encode = encode_preffix + str(bin(nozero_value))[2:]
        elif nozero_value < 0:
                original = str(bin(abs(nozero_value)))[2:]
                encode_suffix = ''.join(['0' if x == '1' else '1' for x in original])
                encode = encode_preffix + encode_suffix
        else:
                encode = encode_preffix
        return encode


In [ ]:
def ac_encode(block):
    # zigzag
    global zigzag_col , zigzag_row
    encode = ''
    zigzag_block = np.empty(8 * 8,dtype=np.int32)
    for index in range(8 * 8):
            zigzag_block[index] = block[zigzag_col[index],zigzag_row[index]]
    last_nozero_index = 0
    for ii in range(8 * 8 - 1,-1,-1):
        if zigzag_block[ii] != 0:
            last_nozero_index = ii 
            break
    assert zigzag_block[last_nozero_index] != 0
    zero_num = 0
    for index in range(1,last_nozero_index +1):
        if zigzag_block[index] ==0 and zero_num < 15:
            zero_num += 1
        else:
            nozero_value  = zigzag_block[index]
            if nozero_value == 0:
                encode_preffix = act_ac_encode(15,0)
                zero_num = 0
            else:
                encode_preffix = act_ac_encode(zero_num , nozero_value)
                zero_num = 0
            encode +=encode_preffix
    if last_nozero_index < 63 and last_nozero_index >=0:
        encode_suffix = act_ac_encode(0,0)
        encode += encode_suffix
    return encode


In [ ]:

codes = ''
for block_index in range(blocks_num):
    if block_index == 0:
        codes += dc_encode(dc_values[block_index],0)
    else:
        codes += dc_encode( dc_values[block_index],dc_values[block_index-1]) 
    codes += ac_encode(ac_values[block_index,:,:])

In [ ]:
code_process = ''
nozero_counter = 0
for code in codes:
    code_process += code
    if code == '0':
        nozero_counter = 0
    elif code == '1':
        nozero_counter += 1
    if nozero_counter >=8 and len(code_process) %8 == 0:
        code_process += '00000000' # 8 个 0
        nozero_counter = 0


In [ ]:
add_zero = (-len(code_process))  % 8
code_process += '0' *add_zero
with open('./lady_py.dat','wb') as f: 
    buffer = 0
    for index ,code in enumerate(code_process):
        if index % 8 == 0 and index > 0:
            f.write(buffer.to_bytes(1,byteorder='little',signed=False))            
            buffer = 0
        buffer += int(code) << (7 - (index % 8))
